In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import requests

In [2]:
df = pd.read_csv("./input/books.csv", error_bad_lines=False)


b'Skipping line 4012: expected 10 fields, saw 11\nSkipping line 5688: expected 10 fields, saw 11\nSkipping line 7056: expected 10 fields, saw 11\nSkipping line 10601: expected 10 fields, saw 11\nSkipping line 10668: expected 10 fields, saw 11\n'


In [3]:
df = df.drop(['isbn','language_code','text_reviews_count'], axis=1)
df = df.rename(columns = {'# num_pages':'num_pages'})
df.head(10)

,bookID,title,authors,average_rating,isbn13,num_pages,ratings_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,652,1944099
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,870,1996446
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,320,5629932
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,352,6267
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,435,2149872
5,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,9780439682589,2690,38872
6,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,9780976540601,152,18
7,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,9780439827607,3342,27410
8,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,9780517226957,815,3602
9,13,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,9780345453747,815,240189


In [7]:
def githubRequestAuthorized(isbn):
    authToken = os.getenv("GOOGLE_BOOKS_API_TOKEN")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    url = "https://www.googleapis.com/books/v1/volumes?q=isbn:{}&key={}".format(isbn,authToken)
    #print("Requesting authorized")
    res = requests.get(url)
    data = res.json()
    return data

In [8]:
def getGenre(isbn):
    url = "https://www.googleapis.com/books/v1/volumes?q=isbn:{}".format(isbn)
    print("Requesting url {}...".format(url))
    res = requests.get(url)
    data = res.json()
    print("done")
    
    return data
#data['ISBN:'+isbn]['details']['genres']


In [9]:
for i in range(len(df)) : 
    isbn = df.loc[i, "isbn13"]
    book = githubRequestAuthorized(isbn)
    try:
        df.loc[i, 'Genre'] = book['items'][0]['volumeInfo']['categories']
        
    except:
        df.loc[i, 'Genre'] = 'Others'


KeyboardInterrupt: 

In [10]:
df.head(100)

,bookID,title,authors,average_rating,isbn13,num_pages,ratings_count,Genre
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,652,1944099,Juvenile Fiction
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,870,1996446,Juvenile Fiction
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,320,5629932,Juvenile Fiction
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,352,6267,Juvenile Fiction
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,435,2149872,Juvenile Fiction
...,...,...,...,...,...,...,...,...
95,152,Anna Karenina,Leo Tolstoy-David Magarshack-Priscilla Meyer,4.04,9780451528612,960,108970,Fiction
96,153,Anna Karenina,Leo Tolstoy-Richard Pevear-Larissa Volokhonsky...,4.04,9780140449174,837,2835,Fiction
97,154,CliffsNotes on Tolstoy's Anna Karenina,Marianne Sturman-Leo Tolstoy,3.89,9780822001836,80,15,Literary Criticism
98,155,Anna Karenina,Leo Tolstoy-Amy Mandelker-Constance Garnett,4.04,9781593080273,803,9362,Fiction


In [11]:
df.to_csv(r'./output/books_enriched.csv')